In [18]:
import nipype.algorithms.modelgen as model   # model generation
from  nipype.interfaces import fsl, ants      
from nipype.interfaces.base import Bunch
import os,json,glob,sys
import numpy
import nibabel
import nilearn.plotting
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd


datadir = '/Users/xinyi/Desktop/data/acc_data'
results_dir = '/Users/xinyi/Desktop/data/output'


from nipype.caching import Memory
mem = Memory(base_dir='.')

if not os.path.exists(results_dir):
    os.mkdir(results_dir)

print('Using data from',datadir)



Using data from /Users/xinyi/Desktop/data/acc_data


In [9]:
img = '/Users/xinyi/Desktop/data/acc_data/t1_mprage_sag_1x1x1_SUB001_20220610_BET_NL_brain.nii'
confounds = pd.read_csv('/Users/xinyi/Desktop/data/regressors/regressors.csv')
confounds

,0,1,2,3,4,5,6,7
0,0.945677,-0.325109,0.050245,-0.998737,0.788609,-0.614895,-0.994951,-0.100363
1,-0.984928,-0.172964,-0.730166,0.683269,0.940167,0.340715,0.066286,-0.997801
2,0.837836,-0.545922,0.381369,0.924423,0.403939,-0.914786,-0.709116,0.705092
3,-0.791305,0.611422,0.843626,0.536932,0.252326,-0.967642,0.423409,0.905939
4,-0.460641,0.887587,0.586321,-0.810079,-0.575620,-0.817717,-0.312455,-0.949933
...,...,...,...,...,...,...,...,...
395,-0.225345,-0.974279,0.142177,0.989841,-0.898439,0.439098,-0.959572,0.281465
396,-0.588610,0.808417,-0.618969,0.785415,-0.307076,-0.951685,-0.233754,-0.972296
397,0.904152,-0.427211,0.887502,-0.460804,0.634981,-0.772528,0.575320,-0.817929
398,-0.930475,-0.366355,0.324959,-0.945728,0.731568,0.681769,-0.788804,-0.614645


In [11]:
confounds['0']

0      0.945677
1     -0.984928
2      0.837836
3     -0.791305
4     -0.460641
         ...   
395   -0.225345
396   -0.588610
397    0.904152
398   -0.930475
399    0.330144
Name: 0, Length: 400, dtype: float64

In [35]:
info = [Bunch(
    conditions = [],
    onsets = [],
    durations = [0],
    regressors = [list(confounds['0']),
                  list(confounds['1']),
                  list(confounds['2']),
                  list(confounds['3']),
                  list(confounds['4']),
                  list(confounds['5']),
                  list(confounds['6']),
                  list(confounds['7']),
                  ],
    regressors_names = ['L0','L1','L2','L3','L4','L5','L6','L7'] 
)]

In [77]:
skip = fsl.ExtractROI
skip_results = skip(in_file='/Users/xinyi/Desktop/data/acc_data/t1_mprage_sag_1x1x1_SUB001_20220610_BET_NL.nii.gz',
                     t_min=4, t_size=-1)

In [78]:
s = model.SpecifyModel()
s.inputs.input_units = 'secs'
s.inputs.time_repetition = 1.7
s.inputs.high_pass_filter_cutoff = 128
s.inputs.functional_runs = img
s.inputs.subject_info = info
specify_model_results = s.run()


In [81]:
reg_0 = ['L0','T', ['L0'],[1]]
reg_1 = ['L1','T', ['L1'],[1]]

contrasts=[reg_0,reg_1]
           
level1design = fsl.model.Level1Design
level1design_results = level1design(interscan_interval = 1.7,
                                    bases = {'none':{'derivs': False}},
                                    session_info = specify_model_results.outputs.session_info,
                                    model_serial_correlations=False,
                                    contrasts=contrasts)


level1design_results.run()


ValueError: not enough values to unpack (expected 4, got 3)

In [82]:
level1design = fsl.model.Level1Design()
level1design.inputs.interscan_interval = 1.7
level1design.inputs.bases = {'none':{'derivs': False}}
level1design.inputs.session_info = specify_model_results.outputs.session_info
level1design.inputs.model_serial_correlations=False
level1design.run() # doctest: +SKIP

ValueError: not enough values to unpack (expected 4, got 3)

In [58]:
modelgen = mem.cache(fsl.model.FEATModel)
modelgen_results = modelgen(fsf_file=level1design_results.outputs.fsf_files,
                            ev_files=level1design_results.outputs.ev_files)
modelgen_results.outputs

AttributeError: 'Level1Design' object has no attribute 'outputs'

In [25]:
fsl.model.Level1Design.help()

Generate FEAT specific files

Examples
--------

>>> level1design = Level1Design()
>>> level1design.inputs.interscan_interval = 2.5
>>> level1design.inputs.bases = {'dgamma':{'derivs': False}}
>>> level1design.inputs.session_info = 'session_info.npz'
>>> level1design.run() # doctest: +SKIP

Inputs::

        [Mandatory]
        interscan_interval: (a float)
                Interscan interval (in secs)
        session_info: (any value)
                Session specific information generated by ``modelgen.SpecifyModel``
        bases: (a dictionary with keys which are 'dgamma' and with values
                  which are a dictionary with keys which are 'derivs' and with
                  values which are a boolean or a dictionary with keys which are
                  'gamma' and with values which are a dictionary with keys which are
                  'derivs' or 'gammasigma' or 'gammadelay' and with values which are
                  any value or a dictionary with keys which are 'custom' 